In [1]:
import environs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common import exceptions
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date
import time
from pprint import pprint
import pandas as pd

In [2]:
# goods_category = input('Укажитенаименование товара:\n')
goods_category = 'Фитнес браслет'
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.implicitly_wait(10)
driver.fullscreen_window()
driver.get('https://www.wildberries.ru/')

In [3]:
elem = driver.find_element(By.CLASS_NAME, 'nav-element__search')
elem.click()

In [4]:
elem = driver.find_element(By.ID, 'mobileSearchInput')
elem.send_keys(goods_category)

In [5]:
elem.send_keys(Keys.ENTER)

In [6]:
# goods = driver.find_elements(By.CLASS_NAME, 'product-card')

In [ ]:
goods_list = []
while True:
    
    goods = driver.find_elements(By.CLASS_NAME, 'product-card')
    
    for i in range(len(goods)):
        if i > 0:
            goods = driver.find_elements(By.CLASS_NAME, 'product-card')
        try:
            goods[i].click()
        except exceptions.ElementNotInteractableException:
            break
        time.sleep(3)
        goods_data = {}
        try:
            brand = driver.find_elements(By.XPATH, '//span[@data-link="text{:product^brandName}"]')[1].text
        except IndexError:
            break
        name = driver.find_elements(By.XPATH, '//span[@data-link="text{:product^goodsName}"]')[1].text
        price = driver.find_element(By.CLASS_NAME, 'price-block__price-wrap').text
        price = price.split('\n')
        seller = driver.find_element(By.CLASS_NAME, 'seller__name').text
        number_sales = driver.find_elements(By.XPATH, '//span[contains(@data-link, "productCardOrderCount")]')[1].text
        link = driver.current_url
        raiting = driver.find_elements(By.XPATH, '//span[@data-link="text{: product^star}"]')[1].text
        reviews_number = driver.find_elements(By.CLASS_NAME, 'same-part-kt__count-review')[1].text
        goods_data['Наименование товара'] = name
        goods_data['Брэнд'] = brand
        try:
            goods_data['Цена со скидкой'] = price[0]
        except IndexError:
            goods_data['Цена со скидкой'] = None
        try:
            goods_data['Цена без скидки'] = price[1]
        except IndexError:
            goods_data['Цена без скидки'] = price[0]
        goods_data['Продавец'] = seller
        goods_data['Количество продаж'] = number_sales
        goods_data['Ссылка на товар'] = link
        goods_data['Рейтинг товара'] = raiting
        goods_data['Количество отзывов'] = reviews_number
        goods_list.append(goods_data)
        next_good = driver.find_element(By.CLASS_NAME, 'sw-slider-kt-mix__btn')
        next_good.click()
        time.sleep(3)
        
    try:
        next_page = driver.find_elements(By.CLASS_NAME, 'pagination__next')[0].get_attribute('href')
        driver.get(next_page)
        time.sleep(3)
    except IndexError:
        break

In [ ]:
len(goods_list)    # mast be > 6500

In [ ]:
df = pd.DataFrame(goods_list) 
df.tail()

In [ ]:
df.to_excel('men_running_shoes.xlsx')